### Configuration

In [12]:
import os
import re
import numpy as np
import pandas as pd
import xarray as xr
import datetime

import mne
from mne.time_frequency import tfr_array_morlet

import utils__config

In [13]:
os.chdir(utils__config.working_directory)
os.getcwd()

'Z:\\Layton\\Sleep_083023'

### Parameters

In [14]:
# S05, Jul 11:
# 11700 (into)
# 15600 (N3 -> N2)
# 16500 (out)
# 18600 (into)
# 21360 (out)

# S05, Jul 12:
# 9900 (into)
# 13950 (out)
# 17700 (into)

# S05, Jul 13:
# 7380 (into)
# 10800 (out)
# 12300 (into)
# 16200 (out)

In [15]:
base_path = 'Cache/Subject05/Jul11/S05_'
fif_path = f'{base_path}Jul11_256hz.fif'
legui_path = 'Cache/Subject05/S05_electrodes.csv'

tpoint = 11700 # transition point between awake/sleep
window = 600 # time before & after the transition point

In [26]:
sw_path = f'{base_path}SW.csv'
spike_path = f'{base_path}spikes.csv'
bad_channel_path = f'{base_path}bad_channels.csv'
best_channel_path = f'{base_path}best_channels.csv'

suffix = f't{tpoint}.csv'

spectro_out = f'{base_path}spectro_{suffix}'
raw_out = f'{base_path}raw_{suffix}'
swa_out = f'{base_path}swa_{suffix}'
beta_out = f'{base_path}beta_{suffix}'
sw_out = f'{base_path}sw_{suffix}'
spike_out = f'{base_path}spike_{suffix}'

In [17]:
sampling_freq = 256 # sampling frequency of the raw FIF file
tfr_decimation = 1 # decimation by Morlet when calculating TFR
mean_bin_division = 128 # division factor to bin samples into mean
rolling_mean_samples = 30 # number of samples over which to calculate rolling mean
#sd_gaussian_window = 10 # larger the number, the more equal weighting in the window
spectrogram_channels = ['C4', 'LOF9'] # these will be your choices for spectrogram 

### Spectrogram

In [18]:
spectro = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)
spectro.pick_channels(ch_names = spectrogram_channels)
spectro_chan_ordered = spectro.ch_names

Opening raw data file Cache/Subject05/Jul11/S05_Jul11_256hz.fif...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_9040\69444552.py:1: RuntimeWarning: This filename (Cache/Subject05/Jul11/S05_Jul11_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  spectro = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


    Range : 0 ... 7249663 =      0.000 ... 28318.996 secs
Ready.
Opening raw data file Z:\Layton\Sleep_083023\Cache\Subject05\Jul11\S05_Jul11_256hz-1.fif...
    Range : 7249664 ... 8921599 =  28319.000 ... 34849.996 secs
Ready.
Reading 0 ... 8921599  =      0.000 ... 34849.996 secs...
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


In [20]:
# Get Morlet TFR
ts_array = spectro.get_data(units = dict(eeg = 'uV'))
ts_array = ts_array[np.newaxis, :, :]

# Get timestamps for each sample
timestamps = spectro.times

# Create time-frequency representation
# using the Morlet Wavelet transform:
freqs = np.arange(1, 26, 1)

tfr = tfr_array_morlet(ts_array, 
                       sfreq = spectro.info['sfreq'],
                       freqs = freqs, 
                       n_cycles = 6.0,
                       zero_mean = False, 
                       use_fft = True, 
                       output = 'power', 
                       decim = tfr_decimation,
                       n_jobs = 4, 
                       verbose = None)

# Remove the dummy dimension (that was required
# due to formatting expectations of MNE Morlet):
tfr = np.squeeze(tfr)

# Convert to Xarray as an intermediate step in
# getting data into Pandas long (2d) format:
tfr = xr.DataArray(tfr,
                   dims = ('channel', 'frequency', 'seconds'),
                   coords = {'channel' : spectro_chan_ordered,
                             'frequency' : freqs,
                             'seconds' : timestamps})

tfr = tfr.to_dataframe(name = 'power').reset_index()

# Frequency-wise log10 normalization
tfr['meanpower'] = tfr.groupby(['channel', 'frequency'])['power'].transform('mean')
tfr['log_meanpower'] = 10 * np.log10(tfr['meanpower'])

tfr['logpower'] = 10 * np.log10(tfr['power'])
tfr['logpower_mean'] = tfr.groupby(['channel', 'frequency'])['logpower'].transform('mean')

tfr['logmpower_freq'] = tfr['logpower'] - tfr['log_meanpower']
tfr['logpower_freq'] = tfr['logpower'] - tfr['logpower_mean']

tfr.drop(columns = ['meanpower', 'log_meanpower', 'logpower_mean'], inplace = True)

# Group by frequency and time bin; define the time bin using floor
# division of seconds by your desired decimation factor (optional step)
tfr['time_bin'] = tfr.groupby(['channel', 'frequency']).cumcount() + 1
tfr['time_bin'] = tfr['time_bin'] // mean_bin_division
tfr = tfr.groupby(['channel', 'frequency', 'time_bin']).mean()
tfr = tfr.reset_index()

# Rolling mean to smooth TFR (optional step)
true_seconds = tfr['seconds']

# tfr = tfr.groupby(['channel', 'frequency']).rolling(window = rolling_mean_samples, 
#                                                     min_periods = 1, 
#                                                     center = True, 
#                                                     win_type = 'gaussian').mean(std = sd_gaussian_window)

tfr = tfr.groupby(['channel', 'frequency']).rolling(window = rolling_mean_samples, 
                                                    min_periods = 1, 
                                                    center = True, 
                                                    win_type = 'gaussian').mean()

tfr = tfr.reset_index()
tfr.drop(columns = ['level_2'], inplace = True)

tfr['seconds'] = true_seconds

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   2 out of   2 | elapsed:   22.4s finished


In [21]:
spectro = tfr.loc[(tfr.seconds >= tpoint - window) & (tfr.seconds <= tpoint + window)]

### Raw Traces (Raw + Beta + SWA)

In [22]:
raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)

# Select only macroelectrodes
raw.pick_types(seeg = True, ecog = True)

# Remove rejected channels
bad_channels = pd.read_csv(bad_channel_path)
bad_channels = bad_channels[bad_channels['channel'].isin(raw.ch_names)]
raw.drop_channels(ch_names = bad_channels['channel'].astype('string'))

# Select channels with the most SW's in each ROI
best_channels = pd.read_csv(best_channel_path)
raw.pick_channels(ch_names = best_channels['Channel'].tolist())

# Save channel names for later use
ch_names = raw.ch_names

Opening raw data file Cache/Subject05/Jul11/S05_Jul11_256hz.fif...


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_9040\1609598292.py:1: RuntimeWarning: This filename (Cache/Subject05/Jul11/S05_Jul11_256hz.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(fif_path, preload = True, verbose = None)


    Range : 0 ... 7249663 =      0.000 ... 28318.996 secs
Ready.
Opening raw data file Z:\Layton\Sleep_083023\Cache\Subject05\Jul11\S05_Jul11_256hz-1.fif...
    Range : 7249664 ... 8921599 =  28319.000 ... 34849.996 secs
Ready.
Reading 0 ... 8921599  =      0.000 ... 34849.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\lal85\AppData\Local\Temp\2\ipykernel_9040\1609598292.py:13: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw.pick_channels(ch_names = best_channels['Channel'].tolist())


In [23]:
# Get low-passed SWA trace
swa = raw.copy()
swa.filter(l_freq = None, h_freq = 4, n_jobs = -1)

# Get high-passed Beta trace
beta = raw.copy()
beta.filter(l_freq = 15, h_freq = 30, n_jobs = -1)

Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 4 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 4.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 5.00 Hz)
- Filter length: 423 samples (1.652 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:    3.1s remaining:   15.7s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    3.2s remaining:    4.6s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:    3.4s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    3.7s finished


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 15 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 15.00
- Lower transition bandwidth: 3.75 Hz (-6 dB cutoff frequency: 13.12 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 227 samples (0.887 s)



[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of  12 | elapsed:    1.2s remaining:    6.3s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:    1.4s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   8 out of  12 | elapsed:    1.6s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:    1.8s finished


<Raw | S05_Jul11_256hz.fif, 12 x 8921600 (34850.0 s), ~816.8 MB, data loaded>

In [24]:
# SWA Trace data
swa = swa.crop(tmin = tpoint - window, tmax = tpoint + window)
swa_seconds = swa.times

swa = swa.get_data()
swa = xr.DataArray(swa,
                   dims = ('channel', 'seconds'),
                   coords = {'channel' : ch_names,
                             'seconds' : swa_seconds})
swa = swa.to_dataframe(name = 'amplitude').reset_index()

swa['seconds'] = swa['seconds'] + (tpoint - window) # re-reference to total recording

# Beta Trace data
beta = beta.crop(tmin = tpoint - window, tmax = tpoint + window)
beta_seconds = beta.times

beta = beta.get_data()
beta = xr.DataArray(beta,
                    dims = ('channel', 'seconds'),
                    coords = {'channel' : ch_names,
                              'seconds' : beta_seconds})
beta = beta.to_dataframe(name = 'amplitude').reset_index()

beta['seconds'] = beta['seconds'] + (tpoint - window)

# Raw Trace data
raw = raw.crop(tmin = tpoint - window, tmax = tpoint + window)
raw_seconds = raw.times

raw = raw.get_data()
raw = xr.DataArray(raw,
                     dims = ('channel', 'seconds'),
                     coords = {'channel' : ch_names,
                               'seconds' : raw_seconds})
raw = raw.to_dataframe(name = 'amplitude').reset_index()

raw['seconds'] = raw['seconds'] + (tpoint - window)

### Slow Waves

In [27]:
# Load Slow Wave data
sw_times = pd.read_csv(sw_path)

# Merge with LeGUI to get channel laterality
legui = pd.read_csv(legui_path)
legui = legui[['elec_label', 'hemisphere', 'roi_1']]
legui.columns = ['Channel', 'laterality', 'region']
sw_times = sw_times.merge(legui, on = 'Channel', how = 'inner')

# Select and rename SW columns
sw_times = sw_times[['ID', 'Channel', 'laterality', 'region', 'Start', 'End',
                     'NegPeak', 'MidCrossing', 'PosPeak']]
sw_times.columns = ['sw_id', 'channel_id', 'sw_laterality', 'sw_region', 'start', 'end',
                    'negative_peak', 'mid_crossing', 'positive_peak']

# Only keep SW's from channels contained in the final Raw selection
sw_times = sw_times[sw_times['channel_id'].isin(ch_names)]

# Cropping
sw_times = sw_times.loc[(sw_times.start >= tpoint - window) & (sw_times.end <= tpoint + window)]
sw_times = sw_times[['channel_id', 'start', 'end']]

### Spikes

In [28]:
# Load Spike data
spikes = pd.read_csv(spike_path)
spikes = spikes[['unit_id', 'seconds', 'unit_laterality', 'unit_region']]

# Select only spikes in the time window
spikes = spikes.loc[(spikes.seconds >= tpoint - window) & (spikes.seconds <= tpoint + window)]

### Export

In [29]:
spectro.to_csv(spectro_out, index = False)
swa.to_csv(swa_out, index = False)
beta.to_csv(beta_out, index = False)
raw.to_csv(raw_out, index = False)
sw_times.to_csv(sw_out, index = False)
spikes.to_csv(spike_out, index = False)